We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a 'db' folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"
print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

Using DB_PATH /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb


We verify that the primary panel aligns with the expected regular-season schedule and row counts across all seasons

In [2]:
required = ["pbp", "schedules", "team_week_panel"]
existing = set(con.execute("SHOW TABLES").df()["name"].tolist())

missing = [t for t in required if t not in existing]

print("Missing", missing)
print("OK" if not missing else "STOP, fix missing tables before continuing")

Missing []
OK


We inspect the play-by-play schema to record which optional columns exist allowing subsequent logic to adapt safely without assuming column presence

In [3]:
pbp_cols_df = con.execute("DESCRIBE pbp").df()
pbp_cols = set(pbp_cols_df["column_name"].tolist())

optional_flags = {
    "return_team": "return_team" in pbp_cols,
    "kickoff_type": "kickoff_type" in pbp_cols,
    "onside_kick": "onside_kick" in pbp_cols,
    "safety_kick": "safety_kick" in pbp_cols,
    "free_kick": "free_kick" in pbp_cols,
    "desc": "desc" in pbp_cols,
}

print("Optional columns present", optional_flags)

pbp_cols_df

Optional columns present {'return_team': True, 'kickoff_type': False, 'onside_kick': False, 'safety_kick': False, 'free_kick': False, 'desc': True}


,column_name,column_type,null,key,default,extra
0,play_id,DOUBLE,YES,None,None,None
1,game_id,VARCHAR,YES,None,None,None
2,old_game_id,VARCHAR,YES,None,None,None
3,home_team,VARCHAR,YES,None,None,None
4,away_team,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
367,xyac_median_yardage,INTEGER,YES,None,None,None
368,xyac_success,DOUBLE,YES,None,None,None
369,xyac_fd,DOUBLE,YES,None,None,None
370,xpass,DOUBLE,YES,None,None,None


We check that team identifiers in the panel align with those in the play-by-play data, ensuring join keys are consistent across datasets to prevent failed matches

In [4]:
panel_cols_df = con.execute("DESCRIBE team_week_panel").df()
panel_cols = set(panel_cols_df["column_name"].tolist())

if "team_id" in panel_cols:
    PANEL_TEAM_COL = "team_id"
elif "team" in panel_cols:
    PANEL_TEAM_COL = "team"
else:
    raise ValueError(f"Could not find a team column in team_week_panel, columns are {sorted(panel_cols)[:50]}")

print("Using panel team column", PANEL_TEAM_COL)

panel_teams = set(
    con.execute(f"SELECT DISTINCT {PANEL_TEAM_COL} AS team_key FROM team_week_panel WHERE {PANEL_TEAM_COL} IS NOT NULL")
      .df()["team_key"]
      .tolist()
)

pbp_posteams = set(
    con.execute("SELECT DISTINCT posteam FROM pbp WHERE posteam IS NOT NULL")
      .df()["posteam"]
      .tolist()
)

intersect = panel_teams.intersection(pbp_posteams)

print("Distinct panel teams", len(panel_teams))
print("Distinct posteam in pbp", len(pbp_posteams))
print("Intersection size", len(intersect))

print("Sample posteam not in panel", sorted(list(pbp_posteams - panel_teams))[:25])
print("Sample panel team not in pbp", sorted(list(panel_teams - pbp_posteams))[:25])


Using panel team column team
Distinct panel teams 35
Distinct posteam in pbp 32
Intersection size 32
Sample posteam not in panel []
Sample panel team not in pbp ['OAK', 'SD', 'STL']


Quick sanity check to confirm that we are not accidentally pulling older schedules seasons that include those codes

In [5]:
con.execute(f"""
SELECT
  {PANEL_TEAM_COL} AS team,
  MIN(season) AS min_season,
  MAX(season) AS max_season,
  COUNT(*) AS rows
FROM team_week_panel
WHERE {PANEL_TEAM_COL} IN ('OAK','SD','STL')
GROUP BY 1
ORDER BY 1
""").df()


,team,min_season,max_season,rows
0,OAK,2012,2019,128
1,SD,2012,2016,80
2,STL,2012,2015,64


We create a minimal special teams base view using safe NULL placeholders for missing optional columns, ensuring downstream SQL remains stable and preventing schema-related breakages

In [6]:
select_return_team = "return_team" if optional_flags["return_team"] else "NULL::VARCHAR AS return_team"
select_kickoff_type = "kickoff_type" if optional_flags["kickoff_type"] else "NULL::VARCHAR AS kickoff_type"
select_onside_kick = "onside_kick" if optional_flags["onside_kick"] else "NULL::INTEGER AS onside_kick"
select_safety_kick = "safety_kick" if optional_flags["safety_kick"] else "NULL::INTEGER AS safety_kick"
select_free_kick = "free_kick" if optional_flags["free_kick"] else "NULL::INTEGER AS free_kick"

select_desc = "\"desc\" AS play_desc" if optional_flags["desc"] else "NULL::VARCHAR AS play_desc"

con.execute(f"""
CREATE OR REPLACE TEMP VIEW pbp_st_base AS
SELECT
  season,
  week,
  game_id,
  posteam,
  defteam,
  {select_return_team},
  play_type,
  {select_kickoff_type},
  {select_onside_kick},
  {select_safety_kick},
  {select_free_kick},
  {select_desc}
FROM pbp
""")

We filter the play-by-play data to include only regular-season games, ensuring the metric calculations align with the existing panel scope.

In [7]:
con.execute("""
CREATE OR REPLACE TEMP VIEW pbp_st_reg AS
SELECT b.*
FROM pbp_st_base b
JOIN schedules s
  ON b.game_id = s.game_id
WHERE s.game_type = 'REG'
""")

We quantify the frequency of missing 'return_team' values for punt and kickoff plays to ensure the accurate interpretation of return component counts

In [8]:
con.execute("""
SELECT
  play_type,
  COUNT(*) AS plays,
  SUM(CASE WHEN return_team IS NULL THEN 1 ELSE 0 END) AS null_return_team,
  AVG(CASE WHEN return_team IS NULL THEN 1.0 ELSE 0.0 END) AS null_return_team_rate
FROM pbp_st_reg
WHERE play_type IN ('punt', 'kickoff')
GROUP BY play_type
ORDER BY play_type
""").df()

,play_type,plays,null_return_team,null_return_team_rate
0,kickoff,34689,658.0,0.018969
1,punt,29463,175.0,0.005940


We build a rare play condition that utilizes explicit kick flags when available and falls back to description keywords when necessary, ensuring robust identification of special teams events across varying data qualities

In [9]:
rare_parts = []

rare_parts.append("kickoff_type IN ('onside', 'safety', 'free_kick', 'free kick', 'safety kick')")
rare_parts.append("onside_kick = 1")
rare_parts.append("safety_kick = 1")
rare_parts.append("free_kick = 1")

if optional_flags["desc"]:
    rare_parts.append("play_desc ILIKE '%onside%'")
    rare_parts.append("play_desc ILIKE '%free kick%'")
    rare_parts.append("play_desc ILIKE '%safety kick%'")
    rare_parts.append("play_desc ILIKE '%fair catch kick%'")
    rare_parts.append("play_desc ILIKE '%free-kick%'")
    rare_parts.append("play_desc ILIKE '%safety-kick%'")

rare_condition = "(" + " OR ".join(rare_parts) + ")"

print("Rare condition SQL")
print(rare_condition)

Rare condition SQL
(kickoff_type IN ('onside', 'safety', 'free_kick', 'free kick', 'safety kick') OR onside_kick = 1 OR safety_kick = 1 OR free_kick = 1 OR play_desc ILIKE '%onside%' OR play_desc ILIKE '%free kick%' OR play_desc ILIKE '%safety kick%' OR play_desc ILIKE '%fair catch kick%' OR play_desc ILIKE '%free-kick%' OR play_desc ILIKE '%safety-kick%')


We create a long events table where each row represents a single special teams event attributed to the correct team for each play component by establishing a granular foundation for team-week aggregation

In [10]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW st_events_long AS

SELECT
  season,
  week,
  posteam AS team_id,
  'punt' AS component
FROM pbp_st_reg
WHERE play_type = 'punt'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  return_team AS team_id,
  'punt_return' AS component
FROM pbp_st_reg
WHERE play_type = 'punt'
  AND return_team IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'kickoff' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  return_team AS team_id,
  'kick_return' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND return_team IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'fg' AS component
FROM pbp_st_reg
WHERE play_type = 'field_goal'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'xp' AS component
FROM pbp_st_reg
WHERE play_type = 'extra_point'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'rare' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND posteam IS NOT NULL
  AND {rare_condition}
""")

We aggregate the long events table into the required component columns for each team-week by transforming granular special teams data into a structured feature set ready for panel integration

In [11]:
con.execute("""
CREATE OR REPLACE TEMP VIEW st_components_team_week AS
SELECT
  season,
  week,
  team_id,

  SUM(CASE WHEN component = 'punt' THEN 1 ELSE 0 END) AS ST_Punt_w,
  SUM(CASE WHEN component = 'punt_return' THEN 1 ELSE 0 END) AS ST_PuntReturn_w,
  SUM(CASE WHEN component = 'kickoff' THEN 1 ELSE 0 END) AS ST_Kickoff_w,
  SUM(CASE WHEN component = 'kick_return' THEN 1 ELSE 0 END) AS ST_KickReturn_w,
  SUM(CASE WHEN component = 'fg' THEN 1 ELSE 0 END) AS ST_FG_w,
  SUM(CASE WHEN component = 'xp' THEN 1 ELSE 0 END) AS ST_XP_w,
  SUM(CASE WHEN component = 'rare' THEN 1 ELSE 0 END) AS ST_Rare_w
FROM st_events_long
GROUP BY season, week, team_id
""")

We compute workload totals exactly as defined in the workflow while keeping all component columns for storage in the panel

In [12]:
con.execute("""
CREATE OR REPLACE TEMP VIEW st_load_team_week AS
SELECT
  season,
  week,
  team_id,

  ST_Punt_w,
  ST_PuntReturn_w,
  ST_Kickoff_w,
  ST_KickReturn_w,
  ST_FG_w,
  ST_XP_w,
  ST_Rare_w,

  (ST_Punt_w + ST_PuntReturn_w + ST_Kickoff_w + ST_KickReturn_w + ST_FG_w + ST_XP_w + ST_Rare_w) AS ST_Load_All_w,

  (ST_Kickoff_w + ST_XP_w + ST_FG_w) AS ST_Load_ScoreLinked_w,

  (ST_Punt_w + ST_PuntReturn_w + ST_KickReturn_w + ST_Rare_w) AS ST_Load_NonScore_w
FROM st_components_team_week
""")

We validate that the workload table is well formed with unique team weeks and reasonable ranges for totals and rare events

In [13]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT season || '-' || week || '-' || team_id) AS distinct_team_weeks,
  MIN(ST_Load_All_w) AS min_all,
  MAX(ST_Load_All_w) AS max_all,
  AVG(ST_Load_All_w) AS avg_all,
  SUM(ST_Rare_w) AS total_rare,
  MAX(ST_Rare_w) AS max_rare
FROM st_load_team_week
""").df()

,rows,distinct_team_weeks,min_all,max_all,avg_all,total_rare,max_rare
0,6782,6782,10.0,40.0,23.21159,855.0,3.0


We cross check kickoffs versus kick returns by week to identify attribution issues or missing "return_team" data

In [14]:
con.execute("""
SELECT
  season,
  week,
  SUM(ST_Kickoff_w) AS total_kickoffs_by_kicking_team,
  SUM(ST_KickReturn_w) AS total_kick_returns_by_return_team,
  SUM(ST_Kickoff_w) - SUM(ST_KickReturn_w) AS kickoff_minus_return
FROM st_load_team_week
GROUP BY season, week
ORDER BY ABS(SUM(ST_Kickoff_w) - SUM(ST_KickReturn_w)) DESC
LIMIT 25
""").df()

,season,week,total_kickoffs_by_kicking_team,total_kick_returns_by_return_team,kickoff_minus_return
0,2016,17,158.0,150.0,8.0
1,2020,9,159.0,151.0,8.0
2,2018,5,161.0,154.0,7.0
3,2019,14,176.0,169.0,7.0
4,2019,13,162.0,155.0,7.0
5,2013,15,186.0,179.0,7.0
6,2015,15,168.0,161.0,7.0
7,2017,6,150.0,143.0,7.0
8,2015,7,148.0,141.0,7.0
9,2020,4,169.0,163.0,6.0


We left join special teams workload components and totals into the team-week panel while preserving all existing rows and filling missing values with zeros

In [15]:
con.execute(f"""
CREATE OR REPLACE TABLE team_week_panel AS
SELECT
  p.*,

  COALESCE(st.ST_Punt_w, 0) AS ST_Punt_w,
  COALESCE(st.ST_PuntReturn_w, 0) AS ST_PuntReturn_w,
  COALESCE(st.ST_Kickoff_w, 0) AS ST_Kickoff_w,
  COALESCE(st.ST_KickReturn_w, 0) AS ST_KickReturn_w,
  COALESCE(st.ST_FG_w, 0) AS ST_FG_w,
  COALESCE(st.ST_XP_w, 0) AS ST_XP_w,
  COALESCE(st.ST_Rare_w, 0) AS ST_Rare_w,

  COALESCE(st.ST_Load_All_w, 0) AS ST_Load_All_w,
  COALESCE(st.ST_Load_ScoreLinked_w, 0) AS ST_Load_ScoreLinked_w,
  COALESCE(st.ST_Load_NonScore_w, 0) AS ST_Load_NonScore_w

FROM team_week_panel p
LEFT JOIN st_load_team_week st
  ON p.season = st.season
 AND p.week = st.week
 AND p.{PANEL_TEAM_COL} = st.team_id
""")

Quick sanity check to confirm row counts are unchanged and verify that workload columns are populated across the full panel

In [16]:
con.execute(f"""
SELECT
  COUNT(*) AS panel_rows,
  COUNT(DISTINCT season || '-' || week || '-' || {PANEL_TEAM_COL}) AS distinct_team_weeks,
  SUM(ST_Load_All_w) AS sum_all,
  SUM(ST_Load_ScoreLinked_w) AS sum_scorelinked,
  SUM(ST_Load_NonScore_w) AS sum_nonscore
FROM team_week_panel
""").df()

,panel_rows,distinct_team_weeks,sum_all,sum_scorelinked,sum_nonscore
0,6782,6782,150833.0,61221.0,89612.0


We compute distribution summaries for each component and total to catch overcounting or misaggregation quickly

In [17]:
con.execute("""
SELECT
  MIN(ST_Punt_w) AS min_punt,
  MAX(ST_Punt_w) AS max_punt,
  AVG(ST_Punt_w) AS avg_punt,

  MIN(ST_PuntReturn_w) AS min_puntret,
  MAX(ST_PuntReturn_w) AS max_puntret,
  AVG(ST_PuntReturn_w) AS avg_puntret,

  MIN(ST_Kickoff_w) AS min_kickoff,
  MAX(ST_Kickoff_w) AS max_kickoff,
  AVG(ST_Kickoff_w) AS avg_kickoff,

  MIN(ST_KickReturn_w) AS min_kickret,
  MAX(ST_KickReturn_w) AS max_kickret,
  AVG(ST_KickReturn_w) AS avg_kickret,

  MIN(ST_FG_w) AS min_fg,
  MAX(ST_FG_w) AS max_fg,
  AVG(ST_FG_w) AS avg_fg,

  MIN(ST_XP_w) AS min_xp,
  MAX(ST_XP_w) AS max_xp,
  AVG(ST_XP_w) AS avg_xp,

  MIN(ST_Rare_w) AS min_rare,
  MAX(ST_Rare_w) AS max_rare,
  AVG(ST_Rare_w) AS avg_rare,

  MIN(ST_Load_All_w) AS min_all,
  MAX(ST_Load_All_w) AS max_all,
  AVG(ST_Load_All_w) AS avg_all
FROM team_week_panel
""").df()

,min_punt,max_punt,avg_punt,min_puntret,max_puntret,avg_puntret,min_kickoff,max_kickoff,avg_kickoff,min_kickret,...,avg_fg,min_xp,max_xp,avg_xp,min_rare,max_rare,avg_rare,min_all,max_all,avg_all
0,0.0,13.0,4.147891,0.0,13.0,4.140519,0.0,11.0,4.897523,0.0,...,1.869213,0.0,10.0,2.260248,0.0,3.0,0.119876,0.0,39.0,22.240195


We inspect the highest workload team-weeks to verify that extreme values are football plausible

In [18]:
con.execute(f"""
SELECT
  season,
  week,
  {PANEL_TEAM_COL} AS team,
  ST_Punt_w,
  ST_PuntReturn_w,
  ST_Kickoff_w,
  ST_KickReturn_w,
  ST_FG_w,
  ST_XP_w,
  ST_Rare_w,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
ORDER BY ST_Load_All_w DESC, season DESC, week DESC, {PANEL_TEAM_COL}
LIMIT 50
""").df()

,season,week,team,ST_Punt_w,ST_PuntReturn_w,ST_Kickoff_w,ST_KickReturn_w,ST_FG_w,ST_XP_w,ST_Rare_w,ST_Load_All_w,ST_Load_ScoreLinked_w,ST_Load_NonScore_w
0,2013,7,IND,9.0,7.0,8.0,7.0,3.0,4.0,1.0,39.0,15.0,24.0
1,2022,15,MIN,6.0,7.0,10.0,10.0,1.0,4.0,0.0,38.0,15.0,23.0
2,2013,1,BAL,10.0,7.0,8.0,8.0,2.0,3.0,0.0,38.0,13.0,25.0
3,2017,6,DET,5.0,7.0,9.0,9.0,2.0,5.0,0.0,37.0,16.0,21.0
4,2013,7,DEN,7.0,9.0,8.0,8.0,2.0,3.0,0.0,37.0,13.0,24.0
5,2017,6,CHI,9.0,7.0,7.0,7.0,2.0,3.0,1.0,36.0,12.0,24.0
6,2014,9,NYG,8.0,6.0,9.0,9.0,1.0,3.0,0.0,36.0,13.0,23.0
7,2013,11,CLE,7.0,9.0,8.0,7.0,2.0,2.0,1.0,36.0,12.0,24.0
8,2012,13,IND,8.0,7.0,8.0,8.0,0.0,5.0,0.0,36.0,13.0,23.0
9,2021,12,DAL,7.0,5.0,9.0,9.0,3.0,2.0,0.0,35.0,14.0,21.0


We spot check recent team-weeks to confirm the merge succeeded and that missing special teams weeks are handled as zeros

In [19]:
con.execute("""
SELECT
  season,
  week,
  team,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
ORDER BY season DESC, week DESC, team
LIMIT 30
""").df()

BinderException: Binder Error: Referenced column "team_id" not found in FROM clause!
Candidate bindings: "team", "game_id", "temp", "stadium_id", "game_date"

LINE 5:   team_id,
          ^